In [5]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats
from ta.momentum import RSIIndicator

In [8]:
from ta.trend import EMAIndicator
def EMA(window_para,to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        result.loc[code == result['code'].shift(window_para),'ema'] = EMAIndicator(close=result['close_weekly'],window=window_para).ema_indicator()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']>result['ema']),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']<result['ema']),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checksma.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para

In [9]:
optimize_result = []
for window_para in range(2,50, 1):
    x=EMA(window_para,to_csv = False)
    optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.45654341384624114
MDD 0.44674585217760326
MDD 0.45465567937670603
MDD 0.445824674050809
MDD 0.4396546775552395
MDD 0.4369269985245503
MDD 0.4348063103382159
MDD 0.4306101377532053
MDD 0.42699707620888394
MDD 0.41479258726562074
MDD 0.398824276550261
MDD 0.393199174321302
MDD 0.3884851706673948
MDD 0.38554505572076103
MDD 0.38551152603321154
MDD 0.3874232379052822
MDD 0.3839426976899444
MDD 0.3840450900638061
MDD 0.38752157627497374
MDD 0.38734658619910023
MDD 0.3921312221455039
MDD 0.3881582435416629
MDD 0.37652284218374654
MDD 0.37029956324220886
MDD 0.3652348210700046
MDD 0.3604715749702007
MDD 0.359283073137583
MDD 0.360869621390227
MDD 0.35768498660460174
MDD 0.3590201887571854
MDD 0.35499497624982285
MDD 0.348039721617216
MDD 0.34646888748414906
MDD 0.3467351558905357
MDD 0.3453361710965574
MDD 0.3437551627022421
MDD 0.3438448939257096
MDD 0.3452261162387415
MDD 0.3461855234329334
MDD 0.34580904421758013
MDD 0.3458350007448178
MDD 0.3424962009050944
MDD 0.3369507139997755
MD

In [10]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1
47,2.218161,49
46,2.187340,48
44,2.159684,46
42,2.158213,44
43,2.156697,45
45,2.156384,47
41,2.146298,43
39,2.125342,41
40,2.124101,42
38,2.100138,40
